In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
from llama_index.core.workflow import Context
from llama_index.llms.ollama import Ollama

In [ ]:
async def add(ctx:Context, a:int, b:int) -> int:
    """Add two numbers."""
    cur_state = await ctx.get("state")
    cur_state["num_fn_calls"] += 1
    await ctx.set("state", cur_state)
    return a + b

async def multiply(ctx:Context, a:int, b:int) -> int:
    """Multiply two numbers"""
    cur_state = await ctx.get("state")
    cur_state["num_fn_calls"] += 1
    await ctx.set("state", cur_state)
    return a * b



In [ ]:
llm = Ollama(
    model="llama3.2:1b",
    request_timeout=1200.0,
    context_window=2048,
    additional_kwargs={
       "num_gpu":1,
       "num_thread":4,
       "nlock":False,
       "numa":False
    }
)

In [ ]:
multiply_agent = ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    llm=llm,
    tools=[multiply]
)

addition_agent = ReActAgent(
    name="add_agent",
    description="Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools=[add],
    llm=llm
)

In [ ]:
workflow = AgentWorkflow(
    agents=[addition_agent, multiply_agent],
    root_agent="multiply_agent",
    initial_state={"num_fn_calls": 0},
    state_prompt="Current state: {state}. User message: {msg}"
)

In [ ]:
ctx = Context(workflow)

In [ ]:
response = await workflow.run(user_msg="Can you multiply 5 and 3?", ctx=ctx)
print(response)

In [ ]:
state = await ctx.get("state")
print(state["num_fn_calls"])